## Mapping the PT-JPL model to an Image collection and upscaling to daily ET (mm/day)

Here we apply the PT-JPL model to an `ee.ImageCollection` and then use an upscaling model to estimate daily ET in mm/day. 

### Meterological data

In this example we will use data form the [ERA5-Land hourly - ECMWF climate reanalysis](https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_HOURLY?hl=en) image collection. Specifically, we will use the air temperature, surface pressure, dewpoint temperature and net radiation bands. 

>RA5-Land is a reanalysis dataset providing a consistent view of the evolution of land variables over several decades at an enhanced resolution compared to ERA5. ERA5-Land has been produced by replaying the land component of the ECMWF ERA5 climate reanalysis. Reanalysis combines model data with observations from across the world into a globally complete and consistent dataset using the laws of physics. Reanalysis produces data that goes several decades back in time, providing an accurate description of the climate of the past. This dataset includes all 50 variables as available on CDS.

In [1]:
import geemap
geemap.ee_initialize()
import ee

date_start = ee.Date('2019-06-01')
date_end = date_start.advance(1, 'year')

## ECMWF collection 
# https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_HOURLY#bands)
bands_to_keep = ee.List(['temperature_2m', 'dewpoint_temperature_2m', 'surface_pressure', 'surface_net_solar_radiation_hourly'])
bands_to_rename = ee.List(['temperature', 'dewpoint_temperature', 'surface_pressure', 'net_radiation_hourly']) # do not change these -> see geeet.common.relative_humidity 

Meteo_collection = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY') \
    .filterDate(date_start, date_end) \
            .select(bands_to_keep, bands_to_rename)
            
# Add relative humidity to the collection: 
from geeet.common import relative_humidity
Meteo_collection = Meteo_collection.map(relative_humidity)  # in % 

# Adjust the units for the geeet.ptjpl model:
def K_to_C(img):
    T = img.select('temperature').subtract(273.15).rename('temperature_C')
    return img.addBands(T)

def Pa_to_KPa(img):
    P = img.select('surface_pressure').divide(1000.0).rename('surface_pressure_KPa')
    return img.addBands(P)

def J_to_W(img):
    R = img.select('net_radiation_hourly').divide(3600.0).rename('net_radiation')
    return img.addBands(R)

Meteo_collection = Meteo_collection.map(K_to_C)
Meteo_collection = Meteo_collection.map(Pa_to_KPa)
Meteo_collection = Meteo_collection.map(J_to_W)
Meteo_collection = Meteo_collection.select('relative_humidity', 'temperature_C', 'surface_pressure_KPa', 'net_radiation')

### Satellite imagery

We will use Sentinel-2 data to calculate NDVI.

In [2]:
S2_collection = ee.ImageCollection('COPERNICUS/S2_SR')\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5)) \
        .sort('system:time_start')

# Let's specify a coordinate so we limit our collection to one S2 tile
lonlat=[38.25, 30.25] 
geom = ee.Geometry.Point(lonlat[0], lonlat[1]) # lon/lat
S2_collection = S2_collection.filterBounds(geom)

def compute_ndvi(img):
    toa = img.divide(10000.)
    NDVI = toa.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return img.addBands(NDVI)

NDVI_collection = S2_collection.map(compute_ndvi).select('NDVI')

We will also calculat `fapar_max`, the maximum values of the photosynthetic active radiation (PAR) absorbed by green vegetation cover. We can use a longer date range to calculate an appropriate value here. 

In [3]:
NDVI_longer = NDVI_collection.filterDate(date_start.advance(-3,'year'), date_start.advance(-1,'year'))
NDVI_study = NDVI_collection.filterDate(date_start, date_end)
from geeet.ptjpl import add_fapar
NDVI_longer = NDVI_longer.map(add_fapar)
fapar_max = NDVI_longer.select('fapar').reduce(ee.Reducer.max())

# Finally, we will add fapar_max as a band to every image in our NDVI_collection:
def add_faparmax(img):
    return img.addBands(fapar_max)

NDVI_study = NDVI_study.map(add_faparmax)

### Merging the satellite and meteo collections

The `NDVI_study` should have 9 images here, while the `Meteo_collection` should have 720 (hourly data for 30 days). The next step is to merge the collections so that we have only 9 images with the correct meteorological data. 

In this example, we note that the ECMWF data has three properties with the datetime: `system:time_start` (in milliseconds), `system:time_end` (in milliseconds), and `system:index` (in the format YYYYDDMMTHH). For now, the latter will be useful to apply a filter. 

We also note that the Sentinel-2 data has two properties named `system:time_start` and `system:time_end` in milliseconds, but they contain the same values. However, we can set a new property with the datetime in the same format as the ECMWF data, and then apply the filter based on this datetime. 

In [4]:
# Set a new property for the S2_collection with the datetime in the same format as the "system:index" property in the Meteo_collection
def add_datetime_property(img):
    d = img.get("system:time_start")
    d_parsed = ee.Date(d).format("yyyyMMdd'T'HH")
    img = img.set('system:datetime', d_parsed)
    return img

NDVI_study = NDVI_study.map(add_datetime_property)

# Now we can make an inner join by filtering this property:
s2_join = ee.Join.inner()
s2_filter = ee.Filter.equals(leftField='system:datetime', rightField='system:index')
s2_innerjoin = ee.ImageCollection(s2_join.apply(NDVI_study, Meteo_collection, s2_filter))
def get_img(feature):
    return ee.Image.cat(feature.get('primary'), feature.get('secondary'))

et_inputs = s2_innerjoin.map(get_img)

### Date and time 

Finally, each image must have two named properties: `doy` (the day of the year) and `time`: the local time of observation. 

In [5]:
# We can obtain both from the system:time_start property, 
# but for the time we need to provide the offset for local time, e.g., for Saudi Arabia this offset is +3. 
time_offset = 3
def add_datetime_properties(img):
    d = ee.Date(img.date())
    doy = d.getRelative('day','year').add(1)  # from 0-based to 1-based. 
    time = d.getRelative('hour', 'day').add(time_offset)  
    img = img.set('doy', doy)
    img = img.set('time', time)
    return img

et_inputs = et_inputs.map(add_datetime_properties)

With the `et_inputs` prepared, we can now map the `geeet.ptjpl_arid` model:

In [6]:
from geeet.ptjpl import ptjpl_arid
et_outputs = et_inputs.map(ptjpl_arid)

Note that the output (LE) is instantaneous ET in $W/m2$. We can use the [Jackson et al. (1983)]("https://doi.org/10.1016/0378-3774(83)90095-1") irradiance model for upscaling this value into daily average ET:
$$
ET_d \approx ET_i R
$$
where R is the ratio of instantaneous to daily radiation and can be obtained as:
$$
R =  \frac{2N}{2.45 \times 10^6 \pi \times sin(\pi \times t/N)}
$$
where N is the number of seconds between sunset and sunrise, and t is the number of seconds between sunrise and the observation time. We also apply a 1/2.45 scaling factor to convert the values to mm/day. 



This is available in the `geeet.common.rad_ratio` function:

In [7]:
from geeet.common import rad_ratio
def add_ET_mm_band(img):
    R = rad_ratio(img)
    LE = img.select('LE')
    ETd = LE.multiply(R).rename('ETd')
    return img.addBands(ETd)

et_outputs = et_outputs.map(add_ET_mm_band)

Note that the computations on the GEE servers are done only upon requesting data, for example, adding the data to a map. I.e., the Map on this next cell will take some time to load. 

In [8]:
Map = geemap.Map(center=[lonlat[1], lonlat[0]], zoom=12)
import geemap.colormaps as cm
ndvi_pal = cm.palettes.ndvi
pal = cm.palettes.YlOrRd
pal_viridis = cm.palettes.viridis
#Map.addLayer(et_inputs.select('relative_humidity'), {'min':0, 'max':100, 'palette':["440154","3a528b","20908d","5dc962","fde725"], 'opacity': 0.8}, 'Relative humidity', False)
#Map.addLayer(et_inputs.select('temperature_C'), {'min':0, 'max':35, 'palette':pal, 'opacity': 0.8}, 'Air temperature (2m)', False)
#Map.addLayer(et_inputs.select('surface_pressure_KPa'), {'min':90, 'max':110, 'palette':pal, 'opacity': 0.8}, 'Surface pressure (KPa)', False)
#Map.addLayer(et_inputs.select('net_radiation'), {'min':0, 'max':500, 'palette':pal, 'opacity': 0.8}, 'Net solar radiation (W/m2)', False)
Map.addLayer(et_inputs.select('NDVI'), {'min':0, 'max':1, 'palette':ndvi_pal, 'opacity': 0.8}, 'NDVI')
#Map.addLayer(et_inputs.select('fapar_max'), {'min':0, 'max':1, 'palette':pal, 'opacity': 0.8}, 'F_aparmax', False)
#Map.addLayer(et_outputs.select(['H']), {'min':0, 'max':500, 'palette':pal_viridis, 'opacity': 0.8}, 'Sensible heat flux', False)
#Map.addLayer(et_outputs.select(['G']), {'min':0, 'max':500, 'palette':pal_viridis, 'opacity': 0.8}, 'Ground heat flux', False)
#Map.addLayer(et_outputs.select(['LEs']), {'min':0, 'max':500, 'palette':pal_viridis, 'opacity': 0.8}, 'LE soil', False)
#Map.addLayer(et_outputs.select(['LEc']), {'min':0, 'max':500, 'palette':pal_viridis, 'opacity': 0.8}, 'LE canopy', False)
#Map.addLayer(et_outputs.select(['LEi']), {'min':0, 'max':500, 'palette':pal_viridis, 'opacity': 0.8}, 'LE interception', False)
Map.addLayer(et_outputs.select(['LE']), {'min':0, 'max':500, 'palette':pal_viridis, 'opacity': 0.8}, 'LE')
Map.addLayer(et_outputs.select(['ETd']), {'min':0, 'max':5, 'palette':pal_viridis, 'opacity': 0.8}, 'Daily ET (mm/day)')
Map

Map(center=[30.25, 38.25], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childre…